Some general purpose scripts.
The code here is very ugly with extensive use of global variables. Do __not__ learn from it :)

# Imports and general routines

In [2]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
import sys

In [5]:
import math
from scipy import stats
from scipy.optimize import curve_fit
from numpy.polynomial import polynomial 

In [6]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [7]:
import itertools

In [8]:
import matplotlib.animation as animation
from IPython.display import HTML
import IPython.display

In [9]:
_my = sys.modules[__name__]

# Timing and plotting time of functions

In [10]:
def is_iterable(obj):
    '''check if obj is iterable'''
    try:
        _ = iter(obj)
        return 1
    except TypeError:
        return 0 

In [11]:
def inputlen(inputs):
    '''return some number that can be considered as the length of the input'''
    MAXINT = 100
    MAXLEN = 4
    MAXSTR = 10
    if not inputs:
        return 0
    if isinstance(inputs,(int,float)) and abs(inputs)< MAXINT:
        return int(abs(inputs))
    if isinstance(inputs,(list,tuple)):
        if len(inputs) < MAXLEN:
            return max(inputlen_(a,MAXINT,MAXSTR) for a in inputs)
        MAXINT = max(MAXINT,len(inputs)*2)
    return inputlen_(inputs,MAXINT,MAXSTR)
    
        

def inputlen_(inputs,maxint=100,maxstr=10):
    if isinstance(inputs,(list,tuple)):
        if all(isinstance(a,(int,float,str)) for a in inputs):
            return len(inputs)
        return sum([inputlen_(a,max(maxint,2*len(inputs)),maxstr) for a in inputs])
    if isinstance(inputs,str):
        return max(len(inputs)-maxstr,1)
    if isinstance(inputs,(int,float)):
        m = int(abs(inputs))
        if m<maxint: return 1
        return 1+mylog2(m-maxint)
    return len(repr(inputs))

In [12]:
# some global variables
program_step_counter = 0
execution_counters = []

# these are temporary global variables needed due to quirks of %timeit magic
temp_func = None
temp_inputs = None
temp_output = None

In [13]:
import itertools
def compare_times(func1,func2,gen,inlen=inputlen):
    gen1, gen2 = walk, walk2 = itertools.tee(gen)
    _,lengths,times1,*_ = timer(func1,gen1,doplot=False,doplot_logscale=False,interpolate=False)
    _,lengths,times2,*_ = timer(func2,gen2,doplot=False,doplot_logscale=False,interpolate=False)
    
    fig, ax1 = plt.subplots()
    lns1 = ax1.plot(lengths, times1, 'g-',label=func1.__name__)
    lns2 = ax1.plot(lengths, times2, 'b-',label=func2.__name__)
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)

    ax1.set_xlabel('Input length')
    ax1.set_ylabel(r'Time ($\mu$s)', color='g')
    plt.show()

In [14]:
import string
import random

def geninputs(gen,n,newseed = False):
    if not newseed:
        random.seed(50943875)
    return (gen(i) for i in range(1,n,int(n/25)+1))

def randomstring(l=10):
        letters = string.ascii_lowercase
        return ''.join(random.choice(letters) for i in range(l))
    
def genstrings(n,l=10):
    return geninputs(lambda k: [randomstring(l) for _ in range(k)],n)

def genintlist(n,l=0):
    if not l: l = n
    def g(k):
        return [random.randint(1,l) for i in range(k)]
    return geninputs(g,n)
    

In [15]:
import sys
class SetTrace(object):
    def __init__(self, func):
        self.func = func

    def __enter__(self):
        sys.settrace(self.func)
        return self

    def __exit__(self, ext_type, exc_value, traceback):
        sys.settrace(None)

def monitor(frame, event, arg):
    if event == "line":
        stepit()
    return monitor

def countlines(func,inputs):
    def g():
        nonlocal func
        nonlocal inputs
        global temp_output
        temp_output = None
        global program_step_counter
        program_step_counter = 0
        with SetTrace(monitor):
                temp_output =  func(*inputs)
        global execution_counters
        execution_counters.append(program_step_counter)

    return g

In [16]:
import functools
import statistics

import inspect 


def temp_func():
    pass


import json
try:
    timefunc_cache = json.load(open("timefunc_cache.json", 'r'))
except (IOError, ValueError):
    timefunc_cache = {}

    

#@functools.lru_cache(maxsize=10000)
def timefunc(func,inputs):
    global execution_counters
    global temp_output
    global cache
    key = (func, repr(inputs)) 
    if key in timefunc_cache:
        return timefunc_cache[key]
    if len(inspect.signature(func).parameters )==1:
        inputs = [inputs]
    temp_func = countlines(func,inputs)
    execution_counters = []
    temp_output = None
    res = %timeit -n4 -r1 -o -q temp_func()
    time =  statistics.median(res.all_runs)
    steps = statistics.median(execution_counters)
    timefunc_cache[key] = (time,steps,temp_output)
    return timefunc_cache[key]
    

In [47]:
def getstats(func,gen,inlen=inputlen,usesteps = True):
    lengths = []
    times = []
    steps = []
    outputs = []
    for inputs in gen:
        # sys.stdout.write("Running for input length "+ str(inlen(inputs))+"..")
        inputlength = inlen(inputs)
        lengths.append(inputlength)
        IPython.display.clear_output()
        sys.stdout.write(f'Profiling {func.__name__}: n={inputlength}')
        sys.stdout.flush()
        time,numsteps,output = timefunc(func,inputs)
        outputs.append(output)
        times.append(time)
        if usesteps:
            steps.append(numsteps)
        else:
            steps.append(int(time* (10**9)))
    IPython.display.clear_output()
    return lengths,times,steps,outputs

In [18]:
def stepit(inc_step=1):
    '''Increment the global counter of number of steps'''
    global program_step_counter
    # sys.stdout.write('*')
    program_step_counter += inc_step

In [50]:
def timer(func,gen,inlen=inputlen,doplot=True,doplot_logscale=False, 
          smoothing_filter=False,interpolate=True , usesteps = True):
    '''time a function on different inputs and plot it
       func: function to run
       gen:  list/tuple/iterable of inputs
       inlen: function to compute the input length (default: inlen)
       doplot: print a plot or not (default: True)
       
       returns a tuple of equal lengthed lists as follows:
       outputs: a list of the outputs
       curve: a function f(n) that estimates the number of steps for input length n
       lengths: a list of the input lengths
       times: a list of the times (in seconds)
       steps: a list of the number of steps (calls to stepit() ) that it took
    
    '''
    lengths,times,steps,outputs = getstats(func,gen,inlen,usesteps)
    #savecache()
    if not lengths: raise ValueError("No inputs given")
    curve = plot_curve(lengths,times,steps,doplot,doplot_logscale,smoothing_filter,interpolate,usesteps)
        
    return curve,lengths,times,steps,outputs
    
    

In [20]:
def integer_hist(map, n = None):
    '''histogram of an array/list of integers'''
    if not n:
        n = max(map)
    plt.hist(map,bins=np.linspace(0,n,n+1))
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

In [21]:
def poly_and_log(lengths,a,b,c):
    '''compute a function of the form f(n)=a*n^b*(log n)^c where c is effectively in {0,1}'''
    # print 'n,a,b,c:',n,a,b,c
    c = max(0,1-(c-1)*(c-1))
    print (f"n={n}")
    ans = np.multiply(a*np.power(n,b),np.power(np.log2(n),c))
    print (f"ans={ans}")
    return ans

In [22]:
def mylog2(n):
    return int(math.log2(max(n,2)))

In [23]:
def myround(x):
    return int(round(5*x)/5)

def fit_poly_log(lengths,steps,smoothing_filter=False, allow_logs = True):
    """Fit to the steps vs lengths curve a function that has the form a*n^b(log n)^c 
       for some very specific choices of a,b,c
    """
    #print ("fitting lengths, steps:", len(lengths),len(steps),lengths[:5],steps[:5])
    if allow_logs:
        funcs = [(lambda n:n**3),(lambda n: (n**2)*mylog2(n)),(lambda n: n**2),(lambda n: n*mylog2(n)),
                     (lambda n: n),(lambda n: mylog2(n)),(lambda n:1)]
        descs = [r"n^3",r"n^2\log n",r"n^2",r"n\log n",r"n",r"\log n","1"]
    else:
        funcs = [(lambda n:n**3),(lambda n: n**2), (lambda n: n), (lambda n:1)]
        descs = [r"n^3",r"n^2",r"n","1"]
        
    if len(lengths)>10:
        offset = sum(lengths[:3])/3
        lengths = lengths[6:]
        steps   = steps[6:]
        
    
    samples = [[float(f(n))  for f in funcs] for n in lengths]

    # print "Features:", features
    # features = np.reshape(features,(-1,len(funcs)))
    if smoothing_filter:
        window_size = int(len(steps)/10)
        window_size = window_size + (window_size % 2) - 1
        if window_size >= 5:
            steps = savitzky_golay(steps, window_size, 3)
    
    # steps = np.reshape(steps,(-1,1))
    
    #print ("Samples, steps", samples[:5],steps[:5])
    #clf = linear_model.LinearRegression()
    clf = linear_model.LassoLars(max_iter=10000,alpha = 1.0, fit_intercept=False, positive=True)
    #clf = linear_model.RANSACRegressor(clf,residual_threshold=np.max(steps))
    clf.fit (samples,steps)
    # return map(lambda c: round(c) if abs(c-round(c))<0.2  c,cls.coefs_)
    # print "Found coefficients:", clf.coef_
    coefs = [myround(i) for i in clf.coef_]
    # print "Rounded coefs:", coefs
    nonzero = np.nonzero(coefs)
    #print(nonzero)
    if not len(nonzero[0]):
        def zerofunc(n): return 0
        return (zerofunc , 1, "0")
    idx = nonzero[0][0]
    coef = coefs[idx]
    #curve_text = "$" + ("" if coef==1 else "{:}".format(coef))+descs[idx]+"$"
    
    return (lambda n: coef*funcs[idx](n) , coef,  descs[idx] )

In [24]:
def fit_exp(lengths,steps):
    EXP_MIN = 0.4
    observations = np.reshape([mylog2(n) for n in steps],(-1,1))
    lengths = np.reshape(lengths,(-1,1))
    # print "lengths, observations:", len(lengths),len(observations), lengths[:5], observations[:5]
    
    clf = linear_model.LinearRegression()
    clf.fit (lengths,observations)
    # return map(lambda c: round(c) if abs(c-round(c))<0.2 else c,cls.coefs_)
    slope = round(clf.coef_[0][0],2)    
    if slope < EXP_MIN:
        return (None,None,None) # this is not an exponential function
    
    scale = int(round(2**clf.intercept_))
    curve_text = r"\exp({0:g}n)".format(slope)
    return (lambda n: scale*(2**(slope*n)) , scale, curve_text)

In [25]:
# Taken from https://scipy-cookbook.readthedocs.io/items/SavitzkyGolay.html 
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')


In [49]:
def plot_curve(lengths,times,steps,doplot=True, doplot_logscale=True, 
               smoothing_filter = False,interpolate=True, usesteps = True):
    '''fit best curve to time vs input length curve'''
    plot_steps = usesteps
    orig_steps = steps
    if plot_steps:
        steps = [max(2,s) for s in steps]
    else:
        steps = [int(t*(10**6)) for t in times]
    
    times = [(10**6)*n for n in times]
    curve_scale = sum(times)/sum(steps) if plot_steps else 10**(-6)
    #print('{0:0.3g} micro-seconds per step'.format(curve_scale))
    curve_text = '(none)'
    exponential = False
    EXP_SLOPE  = 0.4
    MIN_FRACTION = 0.01
    
    
    if interpolate:
        (step_curve, coef,curve_text)= (fit_exp(lengths,steps) if plot_steps else fit_exp(lengths,times))
        if step_curve:
            exponential = True
        else:
            (step_curve,coef,curve_text) = fit_poly_log(lengths,steps, smoothing_filter)

        #print(f"Steps(n) ~ {coef}{curve_text}")

        def curve(n):
            return (curve_scale * step_curve(n)) / (10**6)

        approx = [max(step_curve(n),2) for n in lengths]
    else:
        def curve(n): 
            return 1
    
    if doplot:
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax3 = ax1.twinx()
        lns1 = ax1.plot(lengths, times, 'g-',label='time')
        if plot_steps:
            lns2 = ax2.plot(lengths, orig_steps, 'b-',label='steps')
        if interpolate:
            lns3 = ax3.plot(lengths,approx,'r--',label='curve')
        ax3.get_yaxis().set_visible(False)
        
        # added these three lines
        lns = lns1
        if plot_steps:
            lns += lns2
        if interpolate:
            lns += lns3
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc=0)

        #plt.legend()
        if interpolate:
            ax3.plot(lengths,approx,'r--')
        ax1.set_xlabel('Input length')
        ax1.set_ylabel(r'Time ($\mu$s)', color='g')
        if plot_steps:
            ax2.set_ylabel('Steps', color='b')
        
        if interpolate:
            fig.suptitle(rf'Best fit: ${coef*curve_scale:0.3g} \times {curve_text}$ $\mu$s') 
        plt.show()
        
        if doplot_logscale:
            fig, bx1 = plt.subplots()
            bx2 = bx1.twinx()
            bx3 = bx1.twinx()
            lns1= bx1.plot(lengths, [mylog2(n) for n in times], 'g-',label='log time')
            if plot_steps:
                lns2= bx2.plot(lengths, [mylog2(n) for n in steps], 'b-',label='log steps')
            if interpolate:
                lns3 = bx3.plot(lengths,np.log2(approx),'r--',label='log curve')
            
            lns = lns1
            if plot_steps:
                lns += lns2
            if interpolate:
                lns += lns3

            labs = [l.get_label() for l in lns]
            bx1.legend(lns, labs, loc=0)
            
            bx3.get_yaxis().set_visible(False)        
            bx1.set_xlabel('Input length')
            bx1.set_ylabel('log time', color='g')
            if plot_steps:
                bx2.set_ylabel('log steps', color='b')
            #plt.legend()
            fig.suptitle('Log scale') 
            plt.show()
        
    return curve
        
    
    

In [27]:
def plotfunctions(*L,n=500,logscale=False):
    if not L: return
    if len(L)==1: L += [L[0].__name__ ]
    funcs = [(L[i],L[i+1]) for i in range(0,len(L),2)]
    fig, ax1 = plt.subplots()
    lengths = range(2,n,1)
    lines = []
    i = 0
    
    for (f,desc) in funcs:
        lines += ax1.plot(lengths, [log(f(n)) if logscale else f(n) for n in lengths],color=f"C{i}", label=desc)
        i += 1
    labs = [l.get_label() for l in lines]
    ax1.legend(lines, labs, loc=0)
    ax1.set_xlabel(r'$n$')
    ylabel = r'$\log f(n)$' if logscale else r'$f(n)$'
    ax1.set_ylabel(ylabel, color='g')
    plt.show()



### Testing

In [28]:
# def find(L,s):
#     for i in range(len(L)):
#         if L[i]==s: return i
#     return -1

In [29]:
# def randlist(n):
#     return [randomstring() for i in range(n)]

In [30]:
# random.seed(2020)
# inputs = ((randlist(n),randomstring()) for n in range(10,10000,100))

In [31]:
# timefunc_cache[key]

In [32]:
# c, *_ = timer(find,inputs);

In [33]:
# c(10000)

# Meoization

In [34]:
def memoized(f, return_table = False):
    '''Return a "memoized" version of the function f using internal mem_table.'''
    mem_table = dict()
    def g(*input):
        # step_pc()
        key = repr(input)
        print("Key:",key)
        if key not in mem_table:
            print("not found - now value is", mem_table[key])
            mem_table[key] = f(*input)
            return mem_table[key]
        else:
            return mem_table[key]
    g.non_memoized = f
    return (g,mem_table) if return_table else g

# Graph utilities

In [35]:
import networkx as nx

In [36]:
def isundir(L):
    for i in range(len(L)):
        for j in L[i]:
            if i not in L[j]: return False
    return True

In [37]:
def list_to_graph(L):
    # print "L=",L
    if isundir(L):
        G=nx.Graph()
    else:
        G=nx.DiGraph()
    for i in range(len(L)):
        G.add_node(i)
        # print "i,L[i]",i,L[i]
        for j in L[i]:
            G.add_edge(i,j)
    return G

In [38]:
def nxpos(x,y):
    return np.transpose(np.asarray([x,y],dtype=np.float32))

In [39]:
def grid_layout(G):
    """Takes nx.Graph object which we assume is a two dimensional grid and
       returns a layout for it.
    """
    n = nx.number_of_nodes(G)
    k = int(math.floor(math.sqrt(n)))
    _dict = {}
    for i in range(k):
        for j in range(k):
            _dict[k*i+j] = nxpos(float(i)/float(k),float(j)/float(k))
    t = n-k*k
    for u in range(t):
        _dict[k*k+u]=nxpos(1,float(u)/float(t))
    return _dict

In [40]:
def spring100_layout(G):
        return nx.spring_layout(G,iterations=100)   

In [41]:
def choose_layout_method(my_layout_method):
    L = [nx.shell_layout
         , spring100_layout, nx.spectral_layout]
    S = [grid_layout] # special methods - only chosen if selected by name
    M = [f for f in L+S if f.__name__ == my_layout_method]
    return (M[0] if M else random.choice(L))


In [42]:
def draw_graph(G, my_layout_method=None):
    if not isinstance(G,nx.Graph):
        G = list_to_graph(G)
    f = choose_layout_method(my_layout_method)    
    # print(f.__name__)
    nx.draw(G,pos=f(G))
    # plt.savefig("simple_path.png") # save as png
    plt.show() 

In [43]:
_my.anim_graph = None 
_my.anim_layout = None
_my.anim_colors = None
_my.anim_ax = None
_my.anim_fig = None
_my.anim_layout = None
_my.anim = None
_my.filename = "nofile"

In [44]:
def initialize_animation(G,default_color = 'b', my_layout_method = None, filename = "alg_viz"):
    # print "Graph:", G
    _my.anim_graph = list_to_graph(G)
    # print "Graph (nx):",_my.anim_graph
    _my.anim_fig, _my.anim_ax = plt.subplots()
    f = choose_layout_method(my_layout_method)
    # print "Method =", f
    _my.anim_layout = f(_my.anim_graph)
    # print "Layout=", _my.anim_layout
    _my.anim_colors = [ [default_color for i in range(len(G))]]
    _my.filename = filename

def color(i,new_color):
    last_list = _my.anim_colors[-1]
    new_list = [c for c in last_list]
    new_list[i] = new_color
    _my.anim_colors.append(new_list)

def animate(i):
    nx.draw(_my.anim_graph, pos=_my.anim_layout, node_color = _my.anim_colors[i], ax = _my.anim_ax)

def show_animation(total_time = 10, save=False):
    # plt.rcParams['animation.ffmpeg_path'] =  '/c/Users/boazb/AppData/Local/Continuum/anaconda3/Library/bin/ffmpeg'
    anim = animation.FuncAnimation(_my.anim_fig, animate, 
                                   frames=len(_my.anim_colors), 
                                   interval=(total_time*1000)/len(_my.anim_colors), 
                                   blit=False, repeat=False)
    
    
    _my.anim = anim
    if (save):
        print('saving..')
        anim.save(_my.filename + '.mp4', fps=10, extra_args=['-vcodec', 'libx264'])
    print('rendering..')
    return HTML(anim.to_html5_video())

In [45]:
# G = [[1],[2],[0]]
# initialize_animation(G)
# color(0,'r')
# color(1,'r')
# color(2,'r')
# show_animation()

## Some other useful functions

In [46]:
def log(n):
    return math.log2(max(n,2))